In [1]:
import sagemaker
import boto3
import json
import subprocess
import pandas as pd
from langchain_aws import ChatBedrockConverse, BedrockLLM
from os import environ
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import OpenSearchVectorSearch
from opensearchpy import RequestsHttpConnection, AWSV4SignerAuth
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/willem/.config/sagemaker/config.yaml


In [2]:
# Login AWS
aws_profile = "vrt-analytics-engineer-nonsensitive"
envvars = subprocess.check_output(['aws-vault', 'exec', aws_profile, '--', 'env'])
for envline in envvars.split(b'\n'):
    line = envline.decode('utf8')
    eqpos = line.find('=')
    if eqpos < 4:
        continue
    k = line[0:eqpos]
    v = line[eqpos+1:]
    if k == 'AWS_ACCESS_KEY_ID':
        aws_access_key_id = v
    if k == 'AWS_SECRET_ACCESS_KEY':
        aws_secret_access_key = v
    if k == 'AWS_SESSION_TOKEN':
        aws_session_token = v

session = boto3.Session(
aws_access_key_id, aws_secret_access_key, aws_session_token,region_name="eu-west-1"
)
credentials = session.get_credentials()
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)
auth = AWSV4SignerAuth(credentials, 'eu-west-2', 'aoss')


environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
environ["AWS_SESSION_TOKEN"] = aws_session_token

In [3]:
# LLM endpoint
llm = ChatBedrockConverse(
    region_name="eu-west-2",
    model="meta.llama3-70b-instruct-v1:0",
    temperature=0.6,
    top_p=0.6,
    max_tokens=512
)

/tmp/ipykernel_468837/3432184441.py:2: LangChainBetaWarning: The class `ChatBedrockConverse` is in beta. It is actively being worked on, so the API may change.
  llm = ChatBedrockConverse(


In [4]:
programs = pd.read_csv("programs.csv")

In [5]:
programs.head(60)

,mediacontent_pagetitle_program,brand_contentbrand,mediacontent_media_content_type
0,De krantencommentaren | VRT NU,klara,audio
1,Touché,radio1,audio
2,Alles goed?,radio2,audio
3,Odysseus,radio1,audio
4,Sporza Koers,sporza,audio
5,Ben je nog aan het kijken?,stubru,audio
6,De Rotonde,radio2,audio
7,Waar is zuster Gabrielle?,vrtnws,audio
8,De Laatste Dans,npo,audio
9,De Franse Revolutie,klara,audio


In [6]:
context = programs.head(60).to_json(orient="records")
context= context[1:len(context)-1]

In [22]:
vraag = "ik wil een aflevering van zomerkost zien" 

In [23]:
prompt = """
Je bent een hulpvaardige assistent die Nederlands spreekt.
Op basis van een vraag van een gebruiker moet jij bepalen 
of de gebruiker op zoek is naar een van de programma's die als context zijn meegeven.
Enkel als de naam van het programma letterlijk, of bijna letterlijk, in de vraag van de gebruiker staat mag je dat programma teruggeven.
In elk ander geval moet je zeggen dat er geen programma overeenstemt.
De programma's worden meegegeven in volgend json formaat:

{
"mediacontent_pagetitle_program": "voorbeeld",
"brand_contentbrand":"voorbeeld",
"mediacontent_media_content_type": "voorbeeld"
}

Een voorbeeld: als de gebruiker vraagt naar een episode van Zandman, dan geef je dit programma terug want in de context zit een programma met mediacontent_pagetitle_program Zandman.
Een ander voorbeeld: als de gebruiker vraagt naar een programma over landbouw, dan geef je niet terug. 
Want er is geen enkel programma in de context dat die naam heeft. Er is wel een programma "Voor boer en tuinder" dat waarschijnlijk over landbouw gaat, maar dit geeft je niet terug omdat de naam niet overeenstemt met iets in de vraag van de gebruiker.
"""
prompt += context 
prompt += " " + vraag
llm.invoke(input=prompt)

AIMessage(content='\n\nEr is geen programma dat exact "zomerkost" heet, maar er is wel een programma genaamd "Dagelijkse zomerkost". Dit programma komt overeen met uw vraag.', response_metadata={'ResponseMetadata': {'RequestId': 'd812afc7-8ef8-447d-8592-bf6e637022c1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 10 Sep 2024 13:23:40 GMT', 'content-type': 'application/json', 'content-length': '344', 'connection': 'keep-alive', 'x-amzn-requestid': 'd812afc7-8ef8-447d-8592-bf6e637022c1'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': 1024}}, id='run-ccef3c81-9a97-4d1b-b035-c7ffbb03a41b-0', usage_metadata={'input_tokens': 2029, 'output_tokens': 46, 'total_tokens': 2075})